# The Impact of Remote Work on São Paulo's Real Estate Market: A Spatial Regression Analysis of Price Dynamics
This notebook has the objective of analyzing the impact of remote work on São Paulo's real estate market. The analysis focuses on the relationship between the teleworkability index and property prices in different districts of São Paulo. The study uses spatial regression models to account for spatial dependencies and commuting patterns between districts. The analysis is conducted for the years 2018 to 2021 to understand how the relationship between teleworkability and property prices has evolved over time.

### 1. Importing and Loading Required Libraries

#### 1.1. Installing and Loading Required Packages
This section installs and loads essential R packages for data analysis and visualization. The `stargazer` package is used for creating well-formatted tables. The `spatialreg` package is utilized for spatial regression analysis, and the `sf` package is used for spatial data manipulation. The `flextable` package is used for creating tables. The `spdep` package offers spatial dependence methods. The `tmap` package is used for thematic mapping, and `corrr` is for exploring correlations. The `ggplot2` package is used for visualization, and `ggcorrplot` creates correlation plots. The `car` package offers regression diagnostics, and `HH` provides additional statistical methods. Both `gridExtra` and `cowplot` help arrange multiple plots.



In [ ]:
#install.packages(c("stargazer", "spatialreg", "flextable", "spdep", "corrr", "ggplot2", "ggcorrplot", "car", "HH", "gridExtra", "cowplot", "dplyr"))
library(spatialreg) #for spatial regression analysis
library(sf) #for spatial data manipulation
library(spdep) #for spatial dependence methods
library(corrr) #for exploring correlations
library(ggplot2) #for visualization
library(ggcorrplot) #for correlation plots
library(car) #for regression diagnostics
library(HH) #for additional statistical methods
library(dplyr) #for filter the data by year and remove outliers

---
### 2. Data Preparation and Transformation

#### 2.1. Reading the Main Data Frame
Reading the data from the CSV file and displaying the first few rows to understand the structure of the data.

In [ ]:
data_sell <- read.csv("/Users/halefboukarim/Documents/halef-thesis/web-scraper/resources/clean-data/sell/sell_2018_2021.csv", dec = ".", header = TRUE, sep = ",")

#### 2.2. Convert categorical variables to factors (dummy variables)
Converting categorical variables to factors to use them in the regression model. The variables `gym`, `field_quadra`, `elevator`, `furnished`, and `swimming_pool` are dummy variables and thus need to be converted to factors.

In [ ]:
# Factorizing the categorical variables
data_sell$gym <- as.factor(data_sell$gym)
data_sell$field_quadra <- as.factor(data_sell$field_quadra)
data_sell$elevator <- as.factor(data_sell$elevator)
data_sell$furnished <- as.factor(data_sell$furnished)
data_sell$swimming_pool <- as.factor(data_sell$swimming_pool)

#### 2.3. Convert date to date format
Converting the date column to a date format to filter the data by year. The date column is in the format `dd/mm/yyyy`.

In [ ]:
data_sell$date <- as.Date(data_sell$date, format = "%d/%m/%Y")

#### 2.4. Filter the data by year
Filtering the data by year allows us to run the regression model for each year studied. We can understand the relationship between the variables over time by running the regression for each year. The study will focus on understanding how `price_m2` behaves with the variation of the `teleworkable` index over time.

In [ ]:
# Filtering the data by year to run the regression model for each year studied
data_sell_2018 <- filter(data_sell, year == 2018)
data_sell_2019 <- filter(data_sell, year == 2019)
data_sell_2020 <- filter(data_sell, year == 2020)
data_sell_2021 <- filter(data_sell, year == 2021)

# Set the final data that will be used in the model
data_sell_final <- data_sell
print(nrow(data_sell))
print(nrow(data_sell_final))

#### 2.5. Data Cleaning
Defining a function to remove outliers from a DataFrame based on the interquartile range (IQR) method, specifically targeting the `price_m2` column. It calculates the IQR and uses it to filter out values that fall outside 1.5 times the IQR from the first and third quartiles. The filtered DataFrame is then returned, and the number of removed lines and final rows are printed.

In [ ]:
# Fuction to remove outliers
remove_outliers <- function(df, column_name) {

  # Defining the DataFrame to be cleaned
  df_filtered <- df

  # Removing outliers from the DataFrame using the IQR method
  q1_preco_mes <- quantile(df$price_m2, 0.25, na.rm = TRUE)
  q3_preco_mes <- quantile(df$price_m2, 0.75, na.rm = TRUE)
  iqr_preco_mes <- q3_preco_mes - q1_preco_mes
  lower_bound_preco_mes <- q1_preco_mes - 1.5 * iqr_preco_mes
  upper_bound_preco_mes <- q3_preco_mes + 1.5 * iqr_preco_mes

  # Using the price per square meter as the reference for the filter
  df_filtered <- df_filtered[df_filtered$price_m2 >= lower_bound_preco_mes & df_filtered$price_m2 <= upper_bound_preco_mes,]

  return(df_filtered)
}

df_filtered <- remove_outliers(data_sell_final, "price_m2")
print(paste("Number of removed lines:", nrow(data_sell_final) - nrow(df_filtered)))
print(paste("Number of final rows:", nrow(df_filtered)))

---
### 3. Create the spatial weights matrix
The calculation of the `teleworkability` index assumes that individuals work and reside in the same district. To address this issue, the main model will be re-estimated with a spatial econometric specification. This specification will include a spatial lag of the `teleworkability` index vector, utilizing a spatial weight matrix `commute_matrix` constructed from inter-district commuting flows. This approach will help control for the differing locations of employment and residences.

#### 3.1. Importing the Commute Matrix and Normalizing the Data
This code imports a commute matrix from a CSV file and normalizes it to calculate the proportion of commuters between locations. The normalization transforms each entry to represent the proportion of commuters from one location relative to the total number of commuters from that location. This provides a clearer view of commuting patterns and relative flows between areas.

In [ ]:
# Import the commute matrix
commute_matrix <- read.csv("/Users/halefboukarim/Documents/halef-thesis/occupation/commute_matrix.csv", dec = ".", header = TRUE, sep = ",") %>% as.matrix()

# Normalize the data by dividing each row by the row sum to get the proportion of commuters
commute_matrix_normalized <- sweep(commute_matrix, 1, rowSums(commute_matrix), "/")

#### 3.2. Import the Teleworkability Index Vector for Each Year
This code imports the teleworkability index data for each year from 2018 to 2021 from CSV files. It then converts the `weighted_avg_teleworkable_score` column of each imported matrix into a numeric vector. These vectors represent the teleworkability index for different districts in São Paulo for each respective year.

In [ ]:
# Import the teleworkability index vector for each year and make it a numeric vector
teleworkability_index_2018 <- read.csv("/Users/halefboukarim/Documents/halef-thesis/occupation/teleworkability_district_SP_2018.csv", dec = ".", header = TRUE, sep = ",") %>%  as.matrix()
teleworkability_vector_2018 <- as.numeric(teleworkability_index_2018 [,6])
teleworkability_index_2019 <- read.csv("/Users/halefboukarim/Documents/halef-thesis/occupation/teleworkability_district_SP_2019.csv", dec = ".", header = TRUE, sep = ",") %>% as.matrix()
teleworkability_vector_2019 <- as.numeric(teleworkability_index_2019 [,6])
teleworkability_index_2020 <- read.csv("/Users/halefboukarim/Documents/halef-thesis/occupation/teleworkability_district_SP_2020.csv", dec = ".", header = TRUE, sep = ",") %>% as.matrix()
teleworkability_vector_2020 <- as.numeric(teleworkability_index_2020 [,6])
teleworkability_index_2021 <- read.csv("/Users/halefboukarim/Documents/halef-thesis/occupation/teleworkability_district_SP_2021.csv", dec = ".", header = TRUE, sep = ",") %>% as.matrix()
teleworkability_vector_2021 <- as.numeric(teleworkability_index_2021 [,6])

#### 3.3. Including the spacial lag of the teleworkability index in the model
This code calculates the `spatial lag of the teleworkability index` for each year from 2018 to 2021. It multiplies the `normalized commute matrix` by the `teleworkability vector` for each year, resulting in a new vector that represents the `spatially lagged teleworkability index`, accounting for the influence of commuting patterns between districts.

In [ ]:
spatial_lag_teleworkability_2018 <- commute_matrix_normalized %*% teleworkability_vector_2018
spatial_lag_teleworkability_2019 <- commute_matrix_normalized %*% teleworkability_vector_2019
spatial_lag_teleworkability_2020 <- commute_matrix_normalized %*% teleworkability_vector_2020
spatial_lag_teleworkability_2021 <- commute_matrix_normalized %*% teleworkability_vector_2021

This code combines the `teleworkability index's first and third` columns (ID and year) with the `spatially lagged` teleworkability index for each year from 2018 to 2021. It creates a data frame for each year by first extracting the relevant columns (ID and year) from the original teleworkability index data, then binding these columns with the spatially lagged teleworkability index

In [ ]:
teleworkability_index_2018_first_collunm <- as.data.frame(teleworkability_index_2018 [,c(1,3)])
teleworkability_index_2018_matrix <- cbind(teleworkability_index_2018_first_collunm, spatial_lag_teleworkability_2018)
colnames(teleworkability_index_2018_matrix) <- c("ID", "year", "teleworkability_adj")
teleworkability_index_2019_first_collunm <- as.data.frame(teleworkability_index_2019 [,c(1,3)])
teleworkability_index_2019_matrix <- cbind(teleworkability_index_2019_first_collunm, spatial_lag_teleworkability_2019)
colnames(teleworkability_index_2019_matrix) <- c("ID", "year", "teleworkability_adj")
teleworkability_index_2020_first_collunm <- as.data.frame(teleworkability_index_2020 [,c(1,3)])
teleworkability_index_2020_matrix <- cbind(teleworkability_index_2020_first_collunm, spatial_lag_teleworkability_2020)
colnames(teleworkability_index_2020_matrix) <- c("ID", "year", "teleworkability_adj")
teleworkability_index_2021_first_collunm <- as.data.frame(teleworkability_index_2021 [,c(1,3)])
teleworkability_index_2021_matrix <- cbind(teleworkability_index_2021_first_collunm, spatial_lag_teleworkability_2021)
colnames(teleworkability_index_2021_matrix) <- c("ID", "year", "teleworkability_adj")

This code combines the data frames containing the adjusted teleworkability index for each year from 2018 to 2021 into a single data frame. The `rbind` function is used to bind the rows of the yearly matrices together, and the resulting matrix is then converted into a data frame with `as.data.frame`. This consolidated data frame, `teleworkability_adj_matrix`, allows for comprehensive analysis across all years.

In [ ]:
teleworkability_adj_matrix <- rbind(teleworkability_index_2018_matrix, teleworkability_index_2019_matrix, teleworkability_index_2020_matrix, teleworkability_index_2021_matrix)
teleworkability_adj_matrix <- as.data.frame(teleworkability_adj_matrix)

This code prepares and merges two data frames by matching on the "ID" and "year" columns. First, it converts the "ID" and "year" columns in both `df_filtered` and `teleworkability_adj_matrix` to integer type. Then, it performs a left join to merge `teleworkability_adj_matrix` into `df_filtered` based on the "ID" and "year" columns, resulting in a combined data frame that includes the teleworkability adjusted index.

In [ ]:
df_filtered$ID <- as.integer(df_filtered$ID)
teleworkability_adj_matrix$ID <- as.integer(teleworkability_adj_matrix$ID)
df_filtered$year <- as.integer(df_filtered$year)
teleworkability_adj_matrix$year <- as.integer(teleworkability_adj_matrix$year)
df_filtered <- left_join(df_filtered, teleworkability_adj_matrix, by = c("ID", "year"))

---
### 4. Exploratory Data Analysis (EDA)


In [ ]:
# Select and ensure all columns are numeric
corr_data <- df_filtered[, (names(df_filtered) %in% c("price_m2", "area_m2", "bedrooms", "bathrooms","teleworkability_adj", "metro_dist_km", "delta_cbd_farialima", "inequality_meter", "garage", "avg_salario_medio" ))]

# Convert all columns to numeric
corr_data <- data.frame(lapply(corr_data, function(x) as.numeric(as.character(x))))

# Compute correlation at 2 decimal places
corr_matrix <- round(cor(corr_data, use = "complete.obs"), 2)
ggcorrplot(corr_matrix, hc.order = TRUE, type = "lower", lab = TRUE)

#### 3.1. Defining the Model parameters
This subsection defines the formula for the regression model, specifying `price_m2` as the dependent variable and various property features as independent variables.

In [ ]:
summary(df_filtered)

In [ ]:
params <- (price_real_month ~ teleworkability_adj +
  area_m2 +
  bedrooms +
  bathrooms + 
  metro_dist_km +
  delta_cbd_farialima +
  inequality_meter +
  garage +
  avg_salario_medio)

#data_sell_final_2018 <- filter(data_sell_final, year(date) == 2018)
reg_OLS <- lm(params, data = df_filtered)

#### 3.2 Checking the results for the linear regression model
Checking the results for the linear regression model to understand the relationship between the variables.

In [ ]:
summary(reg_OLS)

#### 3.1.1 Running the diagnostic tests
Running the diagnostic tests to check the normality, heteroskedasticity, and autocorrelation of the residuals

In [ ]:
# Load necessary packages
if (!requireNamespace("nortest", quietly = TRUE)) {
  install.packages("nortest")
}
if (!requireNamespace("lmtest", quietly = TRUE)) {
  install.packages("lmtest")
}
library(nortest)
library(lmtest)

# Perform Shapiro-Wilk test or Anderson-Darling test based on sample size
residuals_best_model <- resid(reg_OLS)
sample_size <- length(residuals_best_model)

if (sample_size >= 3 && sample_size <= 5000) {
  shapiro_test <- shapiro.test(residuals_best_model)
  print(shapiro_test)
  if (shapiro_test$p.value > 0.05) {
    print("Pass: Residuals are normally distributed (fail to reject H0).")
  } else {
    print("Fail: Residuals are not normally distributed (reject H0).")
  }
} else {
  ad_test <- ad.test(residuals_best_model)
  print(ad_test)
  if (ad_test$p.value > 0.05) {
    print("Pass: Residuals are normally distributed (fail to reject H0).")
  } else {
    print("Fail: Residuals are not normally distributed (reject H0).")
  }
}

# Perform Breusch-Pagan test for heteroskedasticity
bp_test <- bptest(reg_OLS)
print(bp_test)
if (bp_test$p.value < 0.05) {
  print("Fail: Heteroskedasticity detected (reject H0).")
} else {
  print("Pass: No heteroskedasticity detected (fail to reject H0).")
}

# Perform Durbin-Watson test for autocorrelation
dw_test <- dwtest(reg_OLS)
print(dw_test)
dw_stat <- dw_test$statistic
if (dw_stat < 1.5) {
  print("Fail: Positive autocorrelation detected.")
} else if (dw_stat > 2.5) {
  print("Fail: Negative autocorrelation detected.")
} else {
  print("Pass: No autocorrelation detected.")
}

In [ ]:
# Calcule os resíduos do modelo
residuals <- residuals(reg_OLS)

# Para calcular Moran's I, precisamos de uma matriz de vizinhança (exemplo com vizinhos mais próximos)
coords <- as.matrix(df_filtered[,c("longitude", "latitude")])
nb <- knn2nb(knearneigh(coords, k=4))
listw <- nb2listw(nb, style="W")

# Teste de Moran's I
moran_test <- moran.test(residuals, listw)
print(moran_test)

In [ ]:
moran.mc(df_filtered$price_real_month, listw, nsim=999)

In [ ]:
lm.morantest(reg_OLS, listw)

#### 3.1.2. Spatial Regression Model

In [ ]:
reg_lag <- lagsarlm(params, data = df_filtered, listw = listw)
summary(reg_lag)

In [ ]:
lag_effects <- impacts(reg_lag, listw = listw, R = 999)
lag_effects

In [ ]:
reg_error <- errorsarlm(params, data = df_filtered, listw = listw)
summary(reg_error)